In [1]:
import pandas as pd
import sqlite3
import json
import web3

from web3 import Web3, HTTPProvider, TestRPCProvider
from solc import compile_source
from web3.contract import ConciseContract

# Registree rocks demo

## 1 Connect to university database

In [2]:
conn = sqlite3.connect('UCTDB.sqlite')
cur = conn.cursor()

## 2 Extract relevant data

In [3]:
students = pd.read_sql_query("""
SELECT ID_student_number.student_number, first_name, last_name, sex, race, date_of_birth
FROM DemographicTable JOIN ID_student_number ON DemographicTable.id_number = ID_student_number.id_number;
""", conn)
course_information = pd.read_sql_query("""
SELECT course_code, course_name, course_credits
FROM ResultsTable;
""", conn).drop_duplicates()
student_grades = pd.read_sql_query("""
SELECT ID_student_number.student_number, ResultsTable.course_code, ResultsTable.course_marks 
FROM ResultsTable JOIN ID_student_number ON ResultsTable.sp_id = ID_student_number.id;
""", conn)

In [4]:
students.head(3)

,student_number,first_name,last_name,sex,race,date_of_birth
0,RAFHUN000000000,Rafael,Hunter,female,other,1998-11-23
1,MARFLE000000001,Marnie,Fletcher,female,other,1996-06-07
2,GRECAL000000002,Gregory,Calvin,male,black,1999-02-12


## 3 Export as csv

### student information

In [5]:
students.to_csv('students.csv', sep='|', index=False)

### Course information

In [6]:
course_information.to_csv('course_info.csv', sep='|', index=False)

### Student performance

In [7]:
grouped_grades = student_grades.groupby('course_code')
for name, group in grouped_grades:
    group.to_csv('{}_grades.csv'.format(name), sep='|', index=False)



## 4 Initialize Blockchain

In [8]:
def compile_contract(contract_file, contract_name):
    contract_sol = compile_source(contract_file) # Compiled source code
    interface = contract_sol['<stdin>:' + contract_name]
    return interface

In [9]:
def deploy_contract(interface):
    # Instantiate and deploy contract
    contract = web3.eth.contract(abi=interface['abi'], bytecode=interface['bin'])

    # Get transaction hash from deployed contract
    tx_hash = contract.deploy(transaction={'from': web3.eth.accounts[0], 'gas': 4100000})

    # Get tx receipt to get contract address
    tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
    contract_address = tx_receipt['contractAddress']

    # Contract instance in concise mode
    contract = web3.eth.contract(abi=interface['abi'], address=contract_address, ContractFactoryClass=ConciseContract)

    return (contract, contract_address)

In [10]:
def import_students_to_blockchain(students, interface):
    student_dict = {}
    print('<<<<<<<< Register Students on Blockchain >>>>>>>>')
    for line in students[1:]:
        (contract, contract_address) = deploy_contract(interface)
        line = line.strip()
        infos = line.split('|')
        contract.setStudentID(infos[0], transact={'from': web3.eth.accounts[0]})
        contract.setFirstName(infos[1], transact={'from': web3.eth.accounts[0]})
        contract.setLastName(infos[2], transact={'from': web3.eth.accounts[0]})
        contract.setSex(infos[3], transact={'from': web3.eth.accounts[0]})
        contract.setRace(infos[4], transact={'from': web3.eth.accounts[0]})
        contract.setDoB(infos[5], transact={'from': web3.eth.accounts[0]})
        student_id = contract.studentID()
        print('Student ID: ', student_id, ' Address: ', contract_address)
        student_dict[student_id] = contract_address
    return student_dict

In [11]:
def import_courses_to_blockchain(courses, interface):
    course_dict = {}
    print('<<<<<<<< Register Courses on Blockchain >>>>>>>>')
    for line in courses[1:]:
        (contract, contract_address) = deploy_contract(interface)
        line = line.strip()
        infos = line.split('|')
        contract.setCode(infos[0], transact={'from': web3.eth.accounts[0]})
        contract.setName(infos[1], transact={'from': web3.eth.accounts[0]})
        contract.setCredits(int(infos[2]), transact={'from': web3.eth.accounts[0]})
        course_code = contract.code()
        print('Course ID: ', course_code, ' Address: ', contract_address)
        course_dict[course_code] = contract_address
    return course_dict

In [12]:
def enroll_students_on_blockchain(student_dict, course_dict, interface):
    print('<<<<<<<< Enroll Students in Courses >>>>>>>>')
    for student_id, student_address in student_dict.items():
        student_contract = web3.eth.contract(abi=interface['abi'], address=student_address, ContractFactoryClass=ConciseContract)
        for course_code, course_address in course_dict.items():
            student_contract.addCourse(course_address, transact={'from': web3.eth.accounts[0]})
        print('Student ID: ', student_id, ' Courses: ', student_contract.getCourseList())


In [13]:
def substitute_id_and_code_with_address(input_filename, output_filename, student_dict, course_dict):
    with open(input_filename, 'r') as myfile:
        grades = myfile.readlines()
    output_grades = []    
    for line in grades[1:]:
        info = line.strip().split('|')
        substitute = student_dict[info[0]]+ '|' + course_dict[info[1]] + '|' + info [2] + '\n'
        output_grades.append(substitute)
    with open(output_filename, 'w') as myfile:
        for line in output_grades:
            myfile.write(line)

In [14]:
# web3.py instance - use local chain
web3 = Web3(HTTPProvider('http://localhost:8545')) # 137.158.107.211
# web3 = Web3(HTTPProvider('http://137.158.107.211:8545')) #

# specify which solidity contract is used
with open('../../playground/test2/contracts/Contracts.sol', 'r') as myfile:
    contract_file = myfile.read() #.replace('\n', '')

student_interface = compile_contract(contract_file, 'Students')
course_interface = compile_contract(contract_file, 'Courses')

with open('students.csv', 'r') as myfile:
    students = myfile.readlines()

student_dict = import_students_to_blockchain(students, student_interface)
with open('student_addresses.csv', 'w') as myfile:
    for key, value in student_dict.items():
        myfile.write(value + '\n')


with open('course_info.csv', 'r') as myfile:
    courses = myfile.readlines()

course_dict = import_courses_to_blockchain(courses, course_interface)
with open('course_addresses.csv', 'w') as myfile:
    for key, value in course_dict.items():
        myfile.write(value + '\n')

enroll_students_on_blockchain(student_dict, course_dict, student_interface)

substitute_id_and_code_with_address('KWS30013C_grades.csv', 'KWS30013C_grades_sub.csv', student_dict, course_dict)
substitute_id_and_code_with_address('WOZ52303W_grades.csv', 'WOZ52303W_grades_sub.csv', student_dict, course_dict)

<<<<<<<< Register Students on Blockchain >>>>>>>>
Student ID:  RAFHUN000000000  Address:  0xA8DD324c459e20BCf667315658cAEaC51dA77F2D
Student ID:  MARFLE000000001  Address:  0xEEBc06851a186e2C7C9A340e62e165155579e277
Student ID:  GRECAL000000002  Address:  0xE0e9D0840fF25848c2a0D4D22b6D1cFA546C5bc9
Student ID:  JASJOS000000003  Address:  0x8821C9eC09aefD1fe294cFed0F6e32aC43dfEba0
Student ID:  ELIMCL000000004  Address:  0xD3b6667DB793F742F9565d11eE4F0980d0631536
Student ID:  MARVAW000000005  Address:  0xE0c42aEAefD57Cdb578759adcdAfc086f9e10Fd4
Student ID:  PAUPRO000000006  Address:  0xf7e31FacA6fA3a10b352df8C945BBAbC34e36C2E
Student ID:  JUDVEG000000007  Address:  0x14D8f234Aa32926A4AFEd52D4BC21F62E3c59cFD
Student ID:  CHAWID000000008  Address:  0x2C13235e720df4f4233692b2Dd4D8FecEB3dB827
Student ID:  KIMSKI000000009  Address:  0xEeb2ABBC65597B76DF039c9913C552A244117976
Student ID:  LISMOR000000010  Address:  0x1B13599F40b879C398E8720a95C64ebDD53DeB74
Student ID:  DANBOW000000011  Address

Student ID:  JUAHIN000000035  Courses:  ['0xbD6C8555B655172f29409dB472671Cb3ba59FC4c', '0xac330069eF5F6AEd07f2FDcB6B13f4A6dc2aE79D', '0x0Ae2F1aAD7e98862f56588e9605B96E4422a5B2d', '0xf04e8D5d777aC2497041676Ac1466c06af351984']
Student ID:  BRIWAL000000015  Courses:  ['0xbD6C8555B655172f29409dB472671Cb3ba59FC4c', '0xac330069eF5F6AEd07f2FDcB6B13f4A6dc2aE79D', '0x0Ae2F1aAD7e98862f56588e9605B96E4422a5B2d', '0xf04e8D5d777aC2497041676Ac1466c06af351984']
Student ID:  AMABEL000000038  Courses:  ['0xbD6C8555B655172f29409dB472671Cb3ba59FC4c', '0xac330069eF5F6AEd07f2FDcB6B13f4A6dc2aE79D', '0x0Ae2F1aAD7e98862f56588e9605B96E4422a5B2d', '0xf04e8D5d777aC2497041676Ac1466c06af351984']
Student ID:  JASJOS000000003  Courses:  ['0xbD6C8555B655172f29409dB472671Cb3ba59FC4c', '0xac330069eF5F6AEd07f2FDcB6B13f4A6dc2aE79D', '0x0Ae2F1aAD7e98862f56588e9605B96E4422a5B2d', '0xf04e8D5d777aC2497041676Ac1466c06af351984']
Student ID:  GERVAS000000021  Courses:  ['0xbD6C8555B655172f29409dB472671Cb3ba59FC4c', '0xac330069eF

# Front End